## Laboratio #3

In [1]:
import pandas as pd

In [2]:
import mysql.connector
from mysql.connector import Error

def create_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [3]:
host = "relational.fit.cvut.cz"
user = "guest"
pwd = "relational"
connection = create_connection(host_name = host, user_name =user ,user_password = pwd)

Connection to MySQL DB successful


In [4]:
query = "SHOW TABLES from employee;"
tables = pd.read_sql(query,connection)
tables

,Tables_in_employee
0,departments
1,dept_emp
2,dept_manager
3,employees
4,salaries
5,titles


In [5]:
query = "select * from employee.employees limit 10;"
employees = pd.read_sql(query,connection)
employees

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
5,10006,1953-04-20,Anneke,Preusig,F,1989-06-02
6,10007,1957-05-23,Tzvetan,Zielinski,F,1989-02-10
7,10008,1958-02-19,Saniya,Kalloufi,M,1994-09-15
8,10009,1952-04-19,Sumant,Peac,F,1985-02-18
9,10010,1963-06-01,Duangkaew,Piveteau,F,1989-08-24


## Parte 1

1.	Obtener el nombre, apellido, salario, título y vigencia del mismo para el empleado 12557. 

In [6]:
query = """
        SELECT 
            A.first_name,
            A.last_name,
            A.emp_no,
            B.title,
            C.from_date,
            C.to_date,
            C.salary
        FROM employee.employees A 
        join employee.titles B on (A.emp_no = B.emp_no)
        join employee.salaries C on (A.emp_no = C.emp_no)
        where A.emp_no = 12557;
        """
subquery = pd.read_sql(query,connection)

In [7]:
subquery

,first_name,last_name,emp_no,title,from_date,to_date,salary
0,Stafford,Kopetz,12557,Senior Engineer,1988-03-20,1989-03-20,41840
1,Stafford,Kopetz,12557,Senior Engineer,1989-03-20,1990-03-20,46021
2,Stafford,Kopetz,12557,Senior Engineer,1990-03-20,1991-03-20,48699
3,Stafford,Kopetz,12557,Senior Engineer,1991-03-20,1992-03-19,51610
4,Stafford,Kopetz,12557,Senior Engineer,1992-03-19,1993-03-19,51351
5,Stafford,Kopetz,12557,Senior Engineer,1993-03-19,1994-03-19,51660
6,Stafford,Kopetz,12557,Senior Engineer,1994-03-19,1995-03-19,52878
7,Stafford,Kopetz,12557,Senior Engineer,1995-03-19,1996-03-18,54053
8,Stafford,Kopetz,12557,Senior Engineer,1996-03-18,1997-03-18,57950
9,Stafford,Kopetz,12557,Senior Engineer,1997-03-18,1998-03-18,58055


2.	Encontrar el último departamento en el que trabajó el empleado con la siguiente información del mismo: nombre, apellido, número de empleado, fecha de nacimiento, código de departamento, nombre del departamento, fecha cuando inició su trabajo en ese departamento y la fecha cuando concluyó su trabajo

In [8]:
query = """
        SELECT 
            A.first_name,
            A.last_name,
            A.birth_date,
            E.*,
            D.to_date,
            B.dept_no,
            C.dept_name
        from (
            SELECT
                emp_no,
                max(from_date) from_date
            FROM employee.salaries
            group by 1) E
        join employee.employees A on (A.emp_no = E.emp_no)
        join employee.dept_emp B on (B.emp_no = E.emp_no)
        join employee.departments C on (B.dept_no = C.dept_no)
        join employee.salaries D on (E.from_date = D.from_date)
        limit 10;
        
        """
subquery = pd.read_sql(query,connection)

In [9]:
subquery

,first_name,last_name,birth_date,emp_no,from_date,to_date,dept_no,dept_name
0,Somnath,Bach,1961-01-24,427374,1986-06-26,1987-06-26,d003,Human Resources
1,Fumitaka,Thombley,1954-03-01,290901,1986-06-26,1987-06-26,d005,Development
2,Tianruo,Montemayor,1963-12-29,75047,1987-06-26,1988-06-25,d009,Customer Service
3,Jongsuk,Harllee,1954-11-20,472895,1988-06-25,1989-06-25,d004,Production
4,Remko,Wegerle,1954-09-03,297321,1988-06-25,1989-06-25,d005,Development
5,Hideo,Radwan,1962-11-04,259837,1988-06-25,1989-06-25,d007,Sales
6,Francoise,Buescher,1960-06-04,239683,1988-06-25,1989-06-25,d008,Research
7,Sanjiv,Casperson,1955-08-01,239403,1988-06-25,1989-06-25,d007,Sales
8,Kazuhisa,Butner,1957-06-04,274332,1989-06-25,1990-06-25,d005,Development
9,Masasuke,Provine,1956-04-17,464415,1990-06-25,1991-06-25,d008,Research


3. Encontrar el salario máximo de cada empleado utilizando la función Partition By con toda la información del empleado y de su salario

In [10]:
query = """
 select *
    from (
        select
            A.emp_no,
            A.birth_date,
            A.first_name,
            A.last_name,
            A.gender,
            B.salary,
            row_number() over (PARTITION BY A.emp_no ORDER BY B.salary DESC) AS seq
        from employee.employees A
        left join employee.salaries B ON B.emp_no = A.emp_no
        group by 1,2,3,4,5,6
    ) A
    where seq = 1
    limit 10;
"""
subquery = pd.read_sql(query,connection)

In [11]:
subquery

,emp_no,birth_date,first_name,last_name,gender,salary,seq
0,10001,1953-09-02,Georgi,Facello,M,88958,1
1,10002,1964-06-02,Bezalel,Simmel,F,72527,1
2,10003,1959-12-03,Parto,Bamford,M,43699,1
3,10004,1954-05-01,Chirstian,Koblick,M,74057,1
4,10005,1955-01-21,Kyoichi,Maliniak,M,94692,1
5,10006,1953-04-20,Anneke,Preusig,F,60098,1
6,10007,1957-05-23,Tzvetan,Zielinski,F,88070,1
7,10008,1958-02-19,Saniya,Kalloufi,M,52668,1
8,10009,1952-04-19,Sumant,Peac,F,94443,1
9,10010,1963-06-01,Duangkaew,Piveteau,F,80324,1


## Parte 2

Se ha cometido un delito y la policía necesita su ayuda. Un detective te dio el informe de la escena del crimen, pero de alguna manera lo perdiste. Recuerdas vagamente que el crimen fue un asesinato que ocurrió en algún momento del 15 de enero de 2018 y que tuvo lugar en SQL City. Empiece por recuperar el informe de la escena del crimen correspondiente de la base de datos del departamento de policía.

In [12]:
import sqlite3
con = sqlite3.connect('sql-murder-mystery.db')

In [13]:
query2 = """
    select name, tbl_name
    from sqlite_master;
    
"""
pd.read_sql(query2, con)

,name,tbl_name
0,crime_scene_report,crime_scene_report
1,drivers_license,drivers_license
2,person,person
3,facebook_event_checkin,facebook_event_checkin
4,interview,interview
5,get_fit_now_member,get_fit_now_member
6,sqlite_autoindex_get_fit_now_member_1,get_fit_now_member
7,get_fit_now_check_in,get_fit_now_check_in
8,income,income
9,solution,solution


In [14]:
query2 = """
    select *
    from crime_scene_report;
    
"""
pd.read_sql(query2, con)

,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,SQL City
...,...,...,...,...
1223,20180430,bribery,\n,Garden Grove
1224,20180430,fraud,‘Why not?’ said the March Hare.\n,Houma
1225,20180430,assault,\n,Fontana
1226,20180501,assault,be NO mistake about it: it was neither more no...,Trenton


In [15]:
#Informacion
query2 = """
    select *
    from crime_scene_report
    where date = 20180115
    and city = "SQL City"
    and type = "murder";
    
"""
pd.read_sql(query2, con)

,date,type,description,city
0,20180115,murder,Security footage shows that there were 2 witne...,SQL City


In [16]:
#Informacion
query2 = """
    select description
    from crime_scene_report
    where date = 20180115
    and city = "SQL City"
    and type = "murder";
    
"""
subquery2 = pd.read_sql(query2, con)

In [17]:
subquery2.iloc[0,0]

'Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".'

Testigos

In [18]:
query2 = """
    select
        id,
        name,
        license_id,
        max(address_number),
        address_street_name
    from person
    where address_street_name like "%Northwestern Dr%";
    
"""
testigo1 = pd.read_sql(query2, con)
testigo1

,id,name,license_id,max(address_number),address_street_name
0,14887,Morty Schapiro,118009,4919,Northwestern Dr


In [19]:
query2 = """
    select *
    from person
    where address_street_name like "%Franklin%"
    and name like "%Annabel%";
    
"""
testigo2 = pd.read_sql(query2, con)
testigo2

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [20]:
#identificando testimonios de los testigos
query2 = """
    select *
    from interview
    where person_id = 14887
    or person_id = 16371;
    
"""
testigos = pd.read_sql(query2, con)
testigos


,person_id,transcript
0,14887,I heard a gunshot and then saw a man run out. ...
1,16371,"I saw the murder happen, and I recognized the ..."


In [21]:
print("Testigo 1")
testigos.iloc[0,1]

Testigo 1


'I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".'

In [22]:
print("Testigo 2")
testigos.iloc[1,1]

Testigo 2


'I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.'

In [23]:
#Buscando la placa que uno de los testigos identificó parcialmente
query2 = """
    select *
    from drivers_license
    where plate_number like "%H42W%"
    and gender = "male";
    
"""
pd.read_sql(query2, con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
1,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [24]:
#Identificando los nombre de los dueños de los vehiculos que coinciden
query2 = """
    select *
    from person
    where license_id = 423327
    or license_id = 664760;
    
"""
pd.read_sql(query2, con)

,id,name,license_id,address_number,address_street_name,ssn
0,51739,Tushar Chandra,664760,312,Phi St,137882671
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [25]:
#Buscando por el numero de membresia parcial del testigo 1
query2 = """
    select *
    from get_fit_now_member
    where id like "48Z%"
    and membership_status = "gold";
    
"""
pd.read_sql(query2, con)

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


In [26]:
#Buscando la entrada de la persona con fecha, 9 de enero
query2 = """
    select *
    from get_fit_now_check_in
    where check_in_date = 20180109
    and membership_id like "%48Z%";
"""
pd.read_sql(query2, con)

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


In [27]:
query2 = """
    select *
    from (
        select *
        from get_fit_now_member
        where id like "48Z%"
        and membership_status = "gold"
    ) A
    join (
        select *
        from get_fit_now_check_in
        where check_in_date = 20180109
        and membership_id like "%48Z%"
    ) B
    on (A.id = B.membership_id)
    join (
        select *
        from person
        where license_id = 423327
        or license_id = 664760
    ) C
    on (A.person_id = C.id)
    ;
    
"""
pd.read_sql(query2, con)

,id,person_id,name,membership_start_date,membership_status,membership_id,check_in_date,check_in_time,check_out_time,id,name,license_id,address_number,address_street_name,ssn
0,48Z55,67318,Jeremy Bowers,20160101,gold,48Z55,20180109,1530,1700,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


La persona que coincide con el vehículo, la membresía oro y la fecha de entrada al gimnasia es Jeremy Bowers, por lo tanto, es el asesino.

In [28]:
#Comprobando el resultado
cursor = con.cursor()
sql = "INSERT INTO solution VALUES (1, 'Jeremy Bowers')"
cursor.execute(sql)
con.commit()
var = pd.read_sql("SELECT value FROM solution", con)
var.iloc[0,0]


"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villian behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."

Se encontró al asesino, pero falta la mente maestra!

In [29]:
#identificando al verdadero villano
query2 = """
    select *
    from interview
    where person_id = 67318;
    
"""
asesino = pd.read_sql(query2, con)
asesino.iloc[0,1]

'I was hired by a woman with a lot of money. I don\'t know her name but I know she\'s around 5\'5" (65") or 5\'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n'

Identificando a la mente maestra.

In [30]:
query2 = """
    select 
        A.*,
        B.*
    from (
        select *
        from drivers_license
        where gender = "female"
        and hair_color = "red"
        and (height >= 65
        and height <= 67)
        and car_make like "%Tesla%"
        and car_model like "%S%"
    ) A
    join person B
    on (A.id = B.license_id)
    join (
        select person_id, event_name, date, count(event_id) asistencias
        from facebook_event_checkin
        where event_name = "SQL Symphony Concert"
        and date like "201712%"
        group by person_id
        having asistencias = 3
    ) C
    on (B.id = C.person_id);
    
"""
pd.read_sql(query2, con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,id,name,license_id,address_number,address_street_name,ssn
0,202298,68,66,green,red,female,500123,Tesla,Model S,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


La persona que coincide con la descripción del asesiono es Miranda Priestly, por lo tanto, es la mente maestra.

In [31]:
#Comprobando el resultado
cursor = con.cursor()
sql = "INSERT INTO solution VALUES (1, 'Miranda Priestly')"
cursor.execute(sql)
con.commit()
var = pd.read_sql("SELECT value FROM solution", con)
var.iloc[0,0]


'Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!'